# Config stuff

In [1]:
import ConnectionConfig as cc
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("DIM_WEATHER")
spark.getActiveSession()

# Create weather dimension

## Step 1: Read weathertypes from csv file

In [3]:
#EXTRACT
df_weathertypes = spark.read.format("csv").option("header",True).option("inferschema",True).load("weathertypes.csv")
df_weathertypes.createOrReplaceTempView("weatherTypes")
df_weathertypes.show()

+----------+-----------------+
|weather_ID|     weather_type|
+----------+-----------------+
|         1|        Aangenaam|
|         2|      Onaangenaam|
|         3|         Neutraal|
|         4|weertype onbekend|
+----------+-----------------+



## Step 2: Add SK to weather dimension

In [5]:
#TRANSFORM
dim_weather = spark.sql("select weather_ID, monotonically_increasing_id() as weather_SK, weather_type \
                         from weatherTypes")

dim_weather.show()

+----------+----------+-----------------+
|weather_ID|weather_SK|     weather_type|
+----------+----------+-----------------+
|         1|         0|        Aangenaam|
|         2|         1|      Onaangenaam|
|         3|         2|         Neutraal|
|         4|         3|weertype onbekend|
+----------+----------+-----------------+



## Step 3: Write to deltatable

In [7]:
#LOAD
dim_weather.write.format("delta").mode("overwrite").saveAsTable("dimWeather")